In [41]:
import pandas as pd
import numpy as np
import geopandas as gpd
import rtree
import math

# Los Angeles

## 0. Import data

In [6]:
# Airbnb listings data
listings_LA = pd.read_csv('data/LosAngeles/filtered_listings_LA.csv')
listings_LA.head()

,Unnamed: 0,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,0.0,109,https://www.airbnb.com/rooms/109,20200213032707,2020-02-13,Amazing bright elegant condo park front *UPGRA...,"*** Unit upgraded with new bamboo flooring, br...","*** Unit upgraded with new bamboo flooring, br...","*** Unit upgraded with new bamboo flooring, br...",none,...,f,f,strict_14_with_grace_period,t,f,1.0,1.0,0.0,0.0,0.02
1,1.0,344,https://www.airbnb.com/rooms/344,20200213032707,2020-02-13,Family perfect;Pool;Near Studios!,This home is perfect for families; aspiring ch...,"Cheerful & comfortable; near studios, amusemen...",This home is perfect for families; aspiring ch...,none,...,t,f,flexible,f,f,1.0,1.0,0.0,0.0,0.18
2,2.0,2708,https://www.airbnb.com/rooms/2708,20200213032707,2020-02-13,Mirrored Mini-Suite with Fireplace - W. Hollywood,Our best memory foam pillows you'll ever sleep...,Flickering fireplace. Blendtec® Designer 625 ...,Our best memory foam pillows you'll ever sleep...,none,...,t,f,strict_14_with_grace_period,f,f,2.0,0.0,2.0,0.0,0.33
3,3.0,2732,https://www.airbnb.com/rooms/2732,20200213032707,2020-02-13,Zen Life at the Beach,NaN,This is a three story townhouse with the follo...,This is a three story townhouse with the follo...,none,...,f,f,strict_14_with_grace_period,f,f,2.0,1.0,1.0,0.0,0.20
4,4.0,2864,https://www.airbnb.com/rooms/2864,20200213032707,2020-02-13,*Upscale Professional Home with Beautiful Studio*,Centrally located.... Furnished with 42 inch S...,The space is furnished with Thomasville furnit...,Centrally located.... Furnished with 42 inch S...,none,...,f,f,strict_14_with_grace_period,f,f,1.0,1.0,0.0,0.0,NaN


In [10]:
#create geodataframe based on longitude and latitude
gdf_listings_LA = gpd.GeoDataFrame(listings_LA, geometry=gpd.points_from_xy(listings_LA.longitude, listings_LA.latitude))
gdf_listings_LA.head()

,Unnamed: 0,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,...,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,geometry
0,0.0,109,https://www.airbnb.com/rooms/109,20200213032707,2020-02-13,Amazing bright elegant condo park front *UPGRA...,"*** Unit upgraded with new bamboo flooring, br...","*** Unit upgraded with new bamboo flooring, br...","*** Unit upgraded with new bamboo flooring, br...",none,...,f,strict_14_with_grace_period,t,f,1.0,1.0,0.0,0.0,0.02,POINT (-118.38494 33.98209)
1,1.0,344,https://www.airbnb.com/rooms/344,20200213032707,2020-02-13,Family perfect;Pool;Near Studios!,This home is perfect for families; aspiring ch...,"Cheerful & comfortable; near studios, amusemen...",This home is perfect for families; aspiring ch...,none,...,f,flexible,f,f,1.0,1.0,0.0,0.0,0.18,POINT (-118.33458 34.16562)
2,2.0,2708,https://www.airbnb.com/rooms/2708,20200213032707,2020-02-13,Mirrored Mini-Suite with Fireplace - W. Hollywood,Our best memory foam pillows you'll ever sleep...,Flickering fireplace. Blendtec® Designer 625 ...,Our best memory foam pillows you'll ever sleep...,none,...,f,strict_14_with_grace_period,f,f,2.0,0.0,2.0,0.0,0.33,POINT (-118.34602 34.09768)
3,3.0,2732,https://www.airbnb.com/rooms/2732,20200213032707,2020-02-13,Zen Life at the Beach,NaN,This is a three story townhouse with the follo...,This is a three story townhouse with the follo...,none,...,f,strict_14_with_grace_period,f,f,2.0,1.0,1.0,0.0,0.20,POINT (-118.48127 34.00475)
4,4.0,2864,https://www.airbnb.com/rooms/2864,20200213032707,2020-02-13,*Upscale Professional Home with Beautiful Studio*,Centrally located.... Furnished with 42 inch S...,The space is furnished with Thomasville furnit...,Centrally located.... Furnished with 42 inch S...,none,...,f,strict_14_with_grace_period,f,f,1.0,1.0,0.0,0.0,NaN,POINT (-118.11397 33.87619)


In [11]:
#read shapefiles of LA Council District
district_LA =gpd.read_file('data/LosAngeles/Council_Districts/geo_export_e9d7a721-6664-49f6-a403-e70598dc8734.shp')
district_LA.head()

,dist_name,district,name,objectid,geometry
0,1 - Gilbert Cedillo,1.0,Gilbert Cedillo,1.0,"POLYGON ((-118.26853 34.04664, -118.26878 34.0..."
1,2 - Paul Krekorian,2.0,Paul Krekorian,2.0,"POLYGON ((-118.33732 34.22103, -118.33455 34.2..."
2,3 - Bob Blumenfield,3.0,Bob Blumenfield,3.0,"POLYGON ((-118.51860 34.20840, -118.51859 34.2..."
3,4 - David Ryu,4.0,David Ryu,4.0,"POLYGON ((-118.43998 34.17939, -118.43998 34.1..."
4,5 - Paul Koretz,5.0,Paul Koretz,5.0,"POLYGON ((-118.51848 34.18661, -118.51849 34.1..."


## 1. Map census data to each district

In [120]:
# import census data
census_LA = pd.read_csv('data/LosAngeles/Economy_Panel_LA.csv')
census_LA.head()

,category,indicator,sub_indicator,locality,gender,council_district,fiscal_year,fy_qtr,calendar_year,cy_qtr,quarter_start,as_of_date,unit_of_measure,unit_text,value,source
0,BUILDING PERMITS,MIX-USE PERMIT VALUES,NaN,LOS ANGELES CITY,NaN,CITY TOTAL,2014.0,2014-03,2014.0,2014-01,1-1-2014,04-11-2017,$,SEASONALLY ADJUSTED AND SMOOTHED,229500000.0,CITY OF LOS ANGELES DEPARTMENT OF BUILDING AND...
1,TAXABLE SALES,TAXABLE SALES BY SECTOR,CLOTHING AND CLOTHING ACCESSORIES STORES,LOS ANGELES COUNTY,NaN,NaN,2013.0,2013-03,2013.0,2013-01,1-1-2013,04-11-2017,$,"THOUSANDS, SEASONALLY ADJUSTED",2485844.7,BOARD OF EQUALIZATION
2,EMPLOYMENT,TOTAL EMPLOYMENT,TOTAL ALL INDUSTRIES,LOS ANGELES CITY,NaN,6,2014.0,2014-03,2014.0,2014-01,1-1-2014,04-11-2017,#,SEASONALLY ADJUSTED,79722.0,"EMPLOYMENT DEVELOPMENT DEPARTMENT, BEACON ECON..."
3,EMPLOYMENT,ESTABLISHMENT CHANGE BY INDUSTRY,MANUFACTURING,LOS ANGELES CITY,NaN,10,2013.0,2013-01,2012.0,2012-03,7-1-2012,04-11-2017,#,NaN,0.0,"EMPLOYMENT DEVELOPMENT DEPARTMENT, BEACON ECON..."
4,EMPLOYMENT,ESTABLISHMENT CHANGE BY INDUSTRY,EDUCATION/HEALTH,LOS ANGELES CITY,NaN,12,2014.0,2014-02,2013.0,2013-04,10-1-2013,04-11-2017,#,NaN,23.0,"EMPLOYMENT DEVELOPMENT DEPARTMENT, BEACON ECON..."


In [126]:
# population in each council district
population_LA = census_LA[(census_LA.indicator=='POPULATION') & (census_LA.calendar_year==2015)] \
                [['indicator','council_district','value']] \
                .sort_values(by='council_district')[:15] \
                .set_index(['council_district'])[['value']]
population_LA

,value
council_district,
1,251630.14
10,238959.80
11,284703.41
12,292429.38
13,255723.30
14,246860.28
15,278403.91
2,254059.22
3,273149.06


In [135]:
# calculate the education indicator (percentage of population with education experience over high school in each council district)
less_than_high_school_LA = pd.pivot_table(census_LA[(census_LA.sub_indicator=='LESS THAN HIGH SCHOOL') & (census_LA.calendar_year==2015)] \
                               ,index='council_district',values='value',aggfunc=np.sum)[:15]
education_LA = pd.DataFrame([[less_than_high_school_LA.index[i],1-less_than_high_school_LA.value[i]/ population_LA.value[i]] \
                             for i in range(len(education_LA))],columns=['council_district','education'])

education_LA['council_district'] = education_LA['council_district'].astype(float)

education_LA = education_LA.set_index(['council_district'])
education_LA

,education
council_district,
1.0,0.731226
10.0,0.825997
11.0,0.948697
12.0,0.928378
13.0,0.812804
14.0,0.774235
15.0,0.816979
2.0,0.866765
3.0,0.875585


In [132]:
# collect the income indicator (median household income in each council district)
income_LA = census_LA[(census_LA.indicator=='MEDIAN HOUSEHOLD INCOME') & (census_LA.calendar_year==2015)] \
                     [['indicator','council_district','value']] \
                     .sort_values(by='council_district')[:15]

income_LA['council_district'] = income_LA['council_district'].astype(float)

income_LA = income_LA.set_index(['council_district'])[['value']] \
                     .rename(columns={'value':'income'})
income_LA

,income
council_district,
1.0,41130.591
10.0,43107.776
11.0,85136.396
12.0,73395.883
13.0,43768.291
14.0,39769.124
15.0,48199.181
2.0,58509.697
3.0,71327.406


In [139]:
# map census data with council districts
district_census_LA = district_LA.merge(income_LA, left_on='district', right_index=True) \
                                .merge(education_LA, left_on='district', right_index=True)
district_census_LA

,dist_name,district,name,objectid,geometry,income,education
0,1 - Gilbert Cedillo,1.0,Gilbert Cedillo,1.0,"POLYGON ((-118.26853 34.04664, -118.26878 34.0...",41130.591,0.731226
1,2 - Paul Krekorian,2.0,Paul Krekorian,2.0,"POLYGON ((-118.33732 34.22103, -118.33455 34.2...",58509.697,0.866765
2,3 - Bob Blumenfield,3.0,Bob Blumenfield,3.0,"POLYGON ((-118.51860 34.20840, -118.51859 34.2...",71327.406,0.875585
3,4 - David Ryu,4.0,David Ryu,4.0,"POLYGON ((-118.43998 34.17939, -118.43998 34.1...",66771.866,0.958282
4,5 - Paul Koretz,5.0,Paul Koretz,5.0,"POLYGON ((-118.51848 34.18661, -118.51849 34.1...",69213.955,0.960307
5,6 - Nury Martinez,6.0,Nury Martinez,6.0,"POLYGON ((-118.36533 34.22870, -118.36417 34.2...",50486.744,0.783831
6,7 - Monica Rodriguez,7.0,Monica Rodriguez,7.0,"POLYGON ((-118.33732 34.22103, -118.33732 34.2...",57166.759,0.794188
7,8 - Marqueece Harris-Dawson,8.0,Marqueece Harris-Dawson,8.0,"POLYGON ((-118.30027 34.03694, -118.29635 34.0...",35758.970,0.798571
8,9 - Curren D. Price Jr.,9.0,Curren D. Price Jr.,9.0,"POLYGON ((-118.23975 34.01479, -118.23976 34.0...",33327.804,0.716395
9,10 - Herb J. Wesson Jr.,10.0,Herb J. Wesson Jr.,10.0,"POLYGON ((-118.28431 34.06371, -118.28420 34.0...",43107.776,0.825997


## 2. Spatial join listings data with census data

In [140]:
# spatial joint
gdf_listings_LA.crs = district_census_LA.crs
listings_with_edu_income = gpd.sjoin(gdf_listings_LA, district_census_LA, how="inner", op="intersects")
listings_with_edu_income.head()

,Unnamed: 0,id,listing_url,scrape_id,last_scraped,name_left,summary,space,description,experiences_offered,...,calculated_host_listings_count_shared_rooms,reviews_per_month,geometry,index_right,dist_name,district,name_right,objectid,income,education
2,2.0,2708,https://www.airbnb.com/rooms/2708,20200213032707,2020-02-13,Mirrored Mini-Suite with Fireplace - W. Hollywood,Our best memory foam pillows you'll ever sleep...,Flickering fireplace. Blendtec® Designer 625 ...,Our best memory foam pillows you'll ever sleep...,none,...,0.0,0.33,POINT (-118.34602 34.09768),3,4 - David Ryu,4.0,David Ryu,4.0,66771.866,0.958282
9,9.0,6931,https://www.airbnb.com/rooms/6931,20200213032707,2020-02-13,Off Sunset Wrap-Around Terr Runyon Canyon View...,Our best memory foam pillows; you'll ever slee...,Decorated with fresh flowers for the Holidays....,Our best memory foam pillows; you'll ever slee...,none,...,0.0,0.14,POINT (-118.34801 34.09521),3,4 - David Ryu,4.0,David Ryu,4.0,66771.866,0.958282
20,20.0,12320,https://www.airbnb.com/rooms/12320,20200213032707,2020-02-13,1930's Spanish Duplex on 9th St,"My place is close to Miracle Mile, Republique,...",Fabulous (Phone number hidden by Airbnb) 's 2B...,"My place is close to Miracle Mile, Republique,...",none,...,0.0,0.54,POINT (-118.34352 34.05864),3,4 - David Ryu,4.0,David Ryu,4.0,66771.866,0.958282
22,22.0,14098,https://www.airbnb.com/rooms/14098,20200213032707,2020-02-13,"Glamour Calls Us A Haven of Peace, Modern Arch...",Roll up the wall at the foot of the bed and le...,RATED #1 FOR short / long term specialty accom...,Roll up the wall at the foot of the bed and le...,none,...,0.0,0.24,POINT (-118.32056 34.11959),3,4 - David Ryu,4.0,David Ryu,4.0,66771.866,0.958282
33,33.0,20786,https://www.airbnb.com/rooms/20786,20200213032707,2020-02-13,MONDRIAN LIKE SKY HIGH STUDIO!,NaN,Need a GREAT spot with superior peace and priv...,Need a GREAT spot with superior peace and priv...,none,...,0.0,0.28,POINT (-118.32044 34.11939),3,4 - David Ryu,4.0,David Ryu,4.0,66771.866,0.958282


## 3. Calculate crime indicator

In [180]:
crime_LA = pd.read_csv('data/LosAngeles/LA_2019_Arrest.csv')
crime_LA

,Arrest Date,Location
0,2019-05-03,"(33.9543, -118.2827)"
1,2019-04-29,"(34.0508, -118.4592)"
2,2019-11-23,"(34.0477, -118.2047)"
3,2019-04-30,"(34.0907, -118.3384)"
4,2019-04-30,"(34.2149, -118.4674)"
...,...,...
93436,2019-04-27,"(33.778, -118.2727)"
93437,2019-04-27,"(34.2156, -118.5975)"
93438,2019-04-27,"(34.0483, -118.2631)"
93439,2019-04-27,"(34.2248, -118.4967)"


In [183]:
# split the latitude and longitude
crime_LA['latitude'] = [float(i.replace('(', '').replace(')', '').split(", ")[0]) for i in crime_LA.Location]
crime_LA['longitude'] = [float(i.replace('(', '').replace(')', '').split(", ")[1]) for i in crime_LA.Location]
crime_LA

,Arrest Date,Location,latitude,longitude
0,2019-05-03,"(33.9543, -118.2827)",33.9543,-118.2827
1,2019-04-29,"(34.0508, -118.4592)",34.0508,-118.4592
2,2019-11-23,"(34.0477, -118.2047)",34.0477,-118.2047
3,2019-04-30,"(34.0907, -118.3384)",34.0907,-118.3384
4,2019-04-30,"(34.2149, -118.4674)",34.2149,-118.4674
...,...,...,...,...
93436,2019-04-27,"(33.778, -118.2727)",33.7780,-118.2727
93437,2019-04-27,"(34.2156, -118.5975)",34.2156,-118.5975
93438,2019-04-27,"(34.0483, -118.2631)",34.0483,-118.2631
93439,2019-04-27,"(34.2248, -118.4967)",34.2248,-118.4967


In [184]:
# create geodataframe based on longitude and latitude
gdf_crime_LA = gpd.GeoDataFrame(crime_LA, geometry=gpd.points_from_xy(crime_LA.longitude, crime_LA.latitude))
gdf_crime_LA.head()

,Arrest Date,Location,latitude,longitude,geometry
0,2019-05-03,"(33.9543, -118.2827)",33.9543,-118.2827,POINT (-118.28270 33.95430)
1,2019-04-29,"(34.0508, -118.4592)",34.0508,-118.4592,POINT (-118.45920 34.05080)
2,2019-11-23,"(34.0477, -118.2047)",34.0477,-118.2047,POINT (-118.20470 34.04770)
3,2019-04-30,"(34.0907, -118.3384)",34.0907,-118.3384,POINT (-118.33840 34.09070)
4,2019-04-30,"(34.2149, -118.4674)",34.2149,-118.4674,POINT (-118.46740 34.21490)


In [146]:
# read shapefiles of LA zip code
zipcode_LA =gpd.read_file('data/LosAngeles/Zip Code Areas/geo_export_0cb487cb-63cc-442d-bf7e-53626237722f.shp')
zipcode_LA.head()

,aland10,awater10,classfp10,display_na,external_i,funcstat10,geoid10,intptlat10,intptlon10,kind,mtfcc10,name,set,slug,zcta5ce10,geometry
0,9071359,0,B5,90001 ZIP Code Tabulation Area (2012),90001,S,90001,+33.9740268,-118.2495088,ZIP Code Tabulation Area (2012),G6350,90001,ZIP Code Tabulation Areas (2012),90001-zip-code-tabulation-area-2012,90001,"POLYGON ((-118.26515 33.97025, -118.26517 33.9..."
1,7930684,0,B5,90002 ZIP Code Tabulation Area (2012),90002,S,90002,+33.9490988,-118.2467371,ZIP Code Tabulation Area (2012),G6350,90002,ZIP Code Tabulation Areas (2012),90002-zip-code-tabulation-area-2012,90002,"POLYGON ((-118.23737 33.95852, -118.23737 33.9..."
2,9197637,403,B5,90003 ZIP Code Tabulation Area (2012),90003,S,90003,+33.9641307,-118.2727831,ZIP Code Tabulation Area (2012),G6350,90003,ZIP Code Tabulation Areas (2012),90003-zip-code-tabulation-area-2012,90003,"POLYGON ((-118.26517 33.98183, -118.26298 33.9..."
3,7894525,0,B5,90004 ZIP Code Tabulation Area (2012),90004,S,90004,+34.0761981,-118.3107225,ZIP Code Tabulation Area (2012),G6350,90004,ZIP Code Tabulation Areas (2012),90004-zip-code-tabulation-area-2012,90004,"POLYGON ((-118.31160 34.06896, -118.33773 34.0..."
4,2807559,0,B5,90005 ZIP Code Tabulation Area (2012),90005,S,90005,+34.0591634,-118.3068924,ZIP Code Tabulation Area (2012),G6350,90005,ZIP Code Tabulation Areas (2012),90005-zip-code-tabulation-area-2012,90005,"MULTIPOLYGON (((-118.29164 34.06179, -118.2899..."


In [185]:
# spatial joint crime data with zip code
gdf_crime_LA.crs = zipcode_LA.crs
crime_zip_LA = gpd.sjoin(gdf_crime_LA, zipcode_LA, how="inner", op="intersects")
crime_zip_LA.head()

,Arrest Date,Location,latitude,longitude,geometry,index_right,aland10,awater10,classfp10,display_na,...,funcstat10,geoid10,intptlat10,intptlon10,kind,mtfcc10,name,set,slug,zcta5ce10
0,2019-05-03,"(33.9543, -118.2827)",33.9543,-118.2827,POINT (-118.28270 33.95430),41,13303666,0,B5,90044 ZIP Code Tabulation Area (2012),...,S,90044,+33.9527248,-118.2919037,ZIP Code Tabulation Area (2012),G6350,90044,ZIP Code Tabulation Areas (2012),90044-zip-code-tabulation-area-2012,90044
13,2019-11-27,"(33.9601, -118.2827)",33.9601,-118.2827,POINT (-118.28270 33.96010),41,13303666,0,B5,90044 ZIP Code Tabulation Area (2012),...,S,90044,+33.9527248,-118.2919037,ZIP Code Tabulation Area (2012),G6350,90044,ZIP Code Tabulation Areas (2012),90044-zip-code-tabulation-area-2012,90044
17,2019-11-21,"(33.9798, -118.2827)",33.9798,-118.2827,POINT (-118.28270 33.97980),41,13303666,0,B5,90044 ZIP Code Tabulation Area (2012),...,S,90044,+33.9527248,-118.2919037,ZIP Code Tabulation Area (2012),G6350,90044,ZIP Code Tabulation Areas (2012),90044-zip-code-tabulation-area-2012,90044
126,2019-04-20,"(33.9774, -118.2827)",33.9774,-118.2827,POINT (-118.28270 33.97740),41,13303666,0,B5,90044 ZIP Code Tabulation Area (2012),...,S,90044,+33.9527248,-118.2919037,ZIP Code Tabulation Area (2012),G6350,90044,ZIP Code Tabulation Areas (2012),90044-zip-code-tabulation-area-2012,90044
127,2019-04-19,"(33.9792, -118.2893)",33.9792,-118.2893,POINT (-118.28930 33.97920),41,13303666,0,B5,90044 ZIP Code Tabulation Area (2012),...,S,90044,+33.9527248,-118.2919037,ZIP Code Tabulation Area (2012),G6350,90044,ZIP Code Tabulation Areas (2012),90044-zip-code-tabulation-area-2012,90044


In [194]:
# count arrests in each zip code as crime indicator
crime_count_LA = pd.pivot_table(crime_zip_LA,values='Location', index='name', aggfunc='count') \
                   .rename(columns={'Location':'crime'})
crime_count_LA

,crime
name,
90001,366
90002,803
90003,2677
90004,1312
90005,603
...,...
91605,1410
91606,1008
91607,308


## 4. Merge crime indicator to the listings data

In [197]:
listings_with_IncEduCri_LA = listings_with_edu_income.merge(crime_count_LA, left_on='zipcode',right_index=True)
listings_with_IncEduCri_LA

,Unnamed: 0,id,listing_url,scrape_id,last_scraped,name_left,summary,space,description,experiences_offered,...,reviews_per_month,geometry,index_right,dist_name,district,name_right,objectid,income,education,crime
2,2.0,2708,https://www.airbnb.com/rooms/2708,20200213032707,2020-02-13,Mirrored Mini-Suite with Fireplace - W. Hollywood,Our best memory foam pillows you'll ever sleep...,Flickering fireplace. Blendtec® Designer 625 ...,Our best memory foam pillows you'll ever sleep...,none,...,0.33,POINT (-118.34602 34.09768),3,4 - David Ryu,4.0,David Ryu,4.0,66771.866,0.958282,641
9,9.0,6931,https://www.airbnb.com/rooms/6931,20200213032707,2020-02-13,Off Sunset Wrap-Around Terr Runyon Canyon View...,Our best memory foam pillows; you'll ever slee...,Decorated with fresh flowers for the Holidays....,Our best memory foam pillows; you'll ever slee...,none,...,0.14,POINT (-118.34801 34.09521),3,4 - David Ryu,4.0,David Ryu,4.0,66771.866,0.958282,641
76,76.0,50005,https://www.airbnb.com/rooms/50005,20200213032707,2020-02-13,Guest House - Nightly or Weekly,Private cozy Guest House in a great location. ...,Retreat in the heart of Hollywood - 1/2 block ...,Private cozy Guest House in a great location. ...,none,...,0.94,POINT (-118.34985 34.09596),3,4 - David Ryu,4.0,David Ryu,4.0,66771.866,0.958282,641
88,88.0,61153,https://www.airbnb.com/rooms/61153,20200213032707,2020-02-13,Gorgeous Furnished Bedroom in Hollywood Hills ...,Great style and comfort in the spacious 4000 s...,One of the most Prestigious and Safest areas i...,Great style and comfort in the spacious 4000 s...,none,...,0.11,POINT (-118.38127 34.11046),3,4 - David Ryu,4.0,David Ryu,4.0,66771.866,0.958282,641
150,150.0,132908,https://www.airbnb.com/rooms/132908,20200213032707,2020-02-13,Spectacular Home With Breath Taking View,A true Mountain Oasis. 180 degree views nestle...,"The view is a killer no doubt about it, as it ...",A true Mountain Oasis. 180 degree views nestle...,none,...,2.15,POINT (-118.37002 34.10456),3,4 - David Ryu,4.0,David Ryu,4.0,66771.866,0.958282,641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38111,38134.0,41823064,https://www.airbnb.com/rooms/41823064,20200213032707,2020-02-13,Super comfortable and cozy!!!,NaN,NaN,NaN,none,...,NaN,POINT (-118.24911 34.00848),8,9 - Curren D. Price Jr.,9.0,Curren D. Price Jr.,9.0,33327.804,0.716395,2335
38113,38136.0,41823240,https://www.airbnb.com/rooms/41823240,20200213032707,2020-02-13,Cozy and Comfortable!,NaN,NaN,NaN,none,...,NaN,POINT (-118.25017 34.00779),8,9 - Curren D. Price Jr.,9.0,Curren D. Price Jr.,9.0,33327.804,0.716395,2335
38661,38684.0,42145957,https://www.airbnb.com/rooms/42145957,20200213032707,2020-02-13,Welcome you to stay at my XX hostel,Brand New House in Downtown LA!! Big Opening D...,There are 2 to 4 bunk beds in each room. Mix G...,Brand New House in Downtown LA!! Big Opening D...,none,...,NaN,POINT (-118.25897 34.02535),8,9 - Curren D. Price Jr.,9.0,Curren D. Price Jr.,9.0,33327.804,0.716395,2335
23023,23049.0,29653320,https://www.airbnb.com/rooms/29653320,20200213032707,2020-02-13,Ollie's Place: Diamond in the rough in central LA,Fully updated 2 bedroom/1 bath home! 5 miles f...,NaN,Fully updated 2 bedroom/1 bath home! 5 miles f...,none,...,0.96,POINT (-118.26157 33.97314),8,9 - Curren D. Price Jr.,9.0,Curren D. Price Jr.,9.0,33327.804,0.716395,366


In [199]:
listings_with_IncEduCri_LA.to_csv('listings_with_IncEduCri_LA.csv')

# New York

# 0. Import listings data merged with income and education indicators

In [253]:
listings_with_edu_income_NYC = pd.read_csv('data/NewYork/nyc_merged.csv')
listings_with_edu_income_NYC['income'] = listings_with_edu_income_NYC['IncomePerCapita']
listings_with_edu_income_NYC['education'] = listings_with_edu_income_NYC['EduAboveHS(%)']
listings_with_edu_income_NYC = listings_with_edu_income_NYC.drop(columns=['IncomePerCapita', 'EduAboveHS(%)'])
listings_with_edu_income_NYC.head()

,Unnamed: 0,Zipcode,id,listing_url,scrape_id,last_scraped,name,summary,space,description,...,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,income,education
0,0,10001,15396.0,https://www.airbnb.com/rooms/15396,2.020021e+13,2020-02-12,Sunny & Spacious Chelsea Apartment,"Modern, spacious, bright and well designed apa...",NaN,"Modern, spacious, bright and well designed apa...",...,strict_14_with_grace_period,f,f,1.0,1.0,0.0,0.0,0.11,77512.0,0.914931
1,1,10001,37042.0,https://www.airbnb.com/rooms/37042,2.020021e+13,2020-02-12,2BR Flex in Hudson Yards near Penn Station,"Close to everything NYC has to offer, the APT ...",Fully furnished bedrooms in a 24/7 doorman ele...,"Close to everything NYC has to offer, the APT ...",...,moderate,f,f,1.0,1.0,0.0,0.0,0.21,77512.0,0.914931
2,2,10001,51572.0,https://www.airbnb.com/rooms/51572,2.020021e+13,2020-02-13,Prime Location in Manhattan,Private Room located on w 30th st right near P...,My space offers a private bedroom in my 2 bedr...,Private Room located on w 30th st right near P...,...,strict_14_with_grace_period,t,t,1.0,0.0,1.0,0.0,3.47,77512.0,0.914931
3,3,10001,98663.0,https://www.airbnb.com/rooms/98663,2.020021e+13,2020-02-12,Groovy NYC Chelsea Pad,Hello! This listing is only available to book ...,Hello! Thanks for checking out my place. I am ...,Hello! This listing is only available to book ...,...,moderate,f,f,1.0,1.0,0.0,0.0,0.18,77512.0,0.914931
4,4,10001,110739.0,https://www.airbnb.com/rooms/110739,2.020021e+13,2020-02-13,Central Nomad/Chelsea Loft Studio - metro dnst...,Long terms discounts available. Spacious and ...,"This is a very central, large loft-like studio...",Long terms discounts available. Spacious and ...,...,strict_14_with_grace_period,f,f,1.0,1.0,0.0,0.0,0.57,77512.0,0.914931


## 1. Calculate crime indicator

In [204]:
crime_NYC = pd.read_csv('data/NewYork/NYC_2019_Arrest.csv')
crime_NYC

,ARREST_DATE,Latitude,Longitude,year
0,09/07/2019,40.674583,-73.930222,2019
1,12/22/2019,40.627978,-73.941647,2019
2,12/31/2019,40.841085,-73.939769,2019
3,12/31/2019,40.596012,-73.742116,2019
4,12/31/2019,40.789803,-73.952289,2019
...,...,...,...,...
214612,01/01/2019,40.766905,-73.963731,2019
214613,01/01/2019,40.673777,-73.763806,2019
214614,01/01/2019,40.674632,-73.939378,2019
214615,01/01/2019,40.645023,-74.077217,2019


In [207]:
# create geodataframe based on longitude and latitude
gdf_crime_NYC = gpd.GeoDataFrame(crime_NYC, geometry=gpd.points_from_xy(crime_NYC.Longitude, crime_NYC.Latitude))
gdf_crime_NYC.head()

,ARREST_DATE,Latitude,Longitude,year,geometry
0,09/07/2019,40.674583,-73.930222,2019,POINT (-73.93022 40.67458)
1,12/22/2019,40.627978,-73.941647,2019,POINT (-73.94165 40.62798)
2,12/31/2019,40.841085,-73.939769,2019,POINT (-73.93977 40.84108)
3,12/31/2019,40.596012,-73.742116,2019,POINT (-73.74212 40.59601)
4,12/31/2019,40.789803,-73.952289,2019,POINT (-73.95229 40.78980)


In [217]:
# read shapefiles of NYC zip code
zipcode_NYC =gpd.read_file('data/NewYork/cb_2018_us_zcta510_500k/cb_2018_us_zcta510_500k.shp')
zipcode_NYC.head()

,ZCTA5CE10,AFFGEOID10,GEOID10,ALAND10,AWATER10,geometry
0,36083,8600000US36083,36083,659750662,5522919,"MULTIPOLYGON (((-85.63225 32.28098, -85.62439 ..."
1,35441,8600000US35441,35441,172850429,8749105,"MULTIPOLYGON (((-87.83287 32.84437, -87.83184 ..."
2,35051,8600000US35051,35051,280236456,5427285,"POLYGON ((-86.74384 33.25002, -86.73802 33.251..."
3,35121,8600000US35121,35121,372736030,5349303,"POLYGON ((-86.58527 33.94743, -86.58033 33.948..."
4,35058,8600000US35058,35058,178039922,3109259,"MULTIPOLYGON (((-86.87884 34.21196, -86.87649 ..."


In [218]:
# spatial joint crime data with zip code
gdf_crime_NYC.crs = zipcode_NYC.crs
crime_zip_NYC = gpd.sjoin(gdf_crime_NYC, zipcode_NYC, how="inner", op="intersects")
crime_zip_NYC.head()

,ARREST_DATE,Latitude,Longitude,year,geometry,index_right,ZCTA5CE10,AFFGEOID10,GEOID10,ALAND10,AWATER10
0,09/07/2019,40.674583,-73.930222,2019,POINT (-73.93022 40.67458),25730,11213,8600000US11213,11213,2824121,0
91,12/31/2019,40.676981,-73.937310,2019,POINT (-73.93731 40.67698),25730,11213,8600000US11213,11213,2824121,0
353,12/30/2019,40.668959,-73.933890,2019,POINT (-73.93389 40.66896),25730,11213,8600000US11213,11213,2824121,0
416,12/30/2019,40.667909,-73.932600,2019,POINT (-73.93260 40.66791),25730,11213,8600000US11213,11213,2824121,0
424,12/30/2019,40.674600,-73.930575,2019,POINT (-73.93057 40.67460),25730,11213,8600000US11213,11213,2824121,0


In [221]:
# count arrests in each zip code as crime indicator
crime_count_NYC = pd.pivot_table(crime_zip_NYC,values='geometry', index='ZCTA5CE10', aggfunc='count') \
                   .rename(columns={'geometry':'crime'})
crime_count_NYC

,crime
ZCTA5CE10,
10001,2846
10002,2721
10003,1817
10004,163
10005,93
...,...
11691,1941
11692,549
11693,724


## 2. Merge crime indicator to the listings data

In [264]:
crime_count_NYC.index = crime_count_NYC.index.astype('int64')

In [265]:
listings_with_IncEduCri_NYC = listings_with_edu_income_NYC.merge(crime_count_NYC, left_on='Zipcode',right_index=True)
listings_with_IncEduCri_NYC

,Unnamed: 0,Zipcode,id,listing_url,scrape_id,last_scraped,name,summary,space,description,...,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,income,education,crime
0,0,10001,15396.0,https://www.airbnb.com/rooms/15396,2.020021e+13,2020-02-12,Sunny & Spacious Chelsea Apartment,"Modern, spacious, bright and well designed apa...",NaN,"Modern, spacious, bright and well designed apa...",...,f,f,1.0,1.0,0.0,0.0,0.11,77512.0,0.914931,2846
1,1,10001,37042.0,https://www.airbnb.com/rooms/37042,2.020021e+13,2020-02-12,2BR Flex in Hudson Yards near Penn Station,"Close to everything NYC has to offer, the APT ...",Fully furnished bedrooms in a 24/7 doorman ele...,"Close to everything NYC has to offer, the APT ...",...,f,f,1.0,1.0,0.0,0.0,0.21,77512.0,0.914931,2846
2,2,10001,51572.0,https://www.airbnb.com/rooms/51572,2.020021e+13,2020-02-13,Prime Location in Manhattan,Private Room located on w 30th st right near P...,My space offers a private bedroom in my 2 bedr...,Private Room located on w 30th st right near P...,...,t,t,1.0,0.0,1.0,0.0,3.47,77512.0,0.914931,2846
3,3,10001,98663.0,https://www.airbnb.com/rooms/98663,2.020021e+13,2020-02-12,Groovy NYC Chelsea Pad,Hello! This listing is only available to book ...,Hello! Thanks for checking out my place. I am ...,Hello! This listing is only available to book ...,...,f,f,1.0,1.0,0.0,0.0,0.18,77512.0,0.914931,2846
4,4,10001,110739.0,https://www.airbnb.com/rooms/110739,2.020021e+13,2020-02-13,Central Nomad/Chelsea Loft Studio - metro dnst...,Long terms discounts available. Spacious and ...,"This is a very central, large loft-like studio...",Long terms discounts available. Spacious and ...,...,f,f,1.0,1.0,0.0,0.0,0.57,77512.0,0.914931,2846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49486,49486,11694,36417725.0,https://www.airbnb.com/rooms/36417725,2.020021e+13,2020-02-13,Ocenfront Glamping in Queens,NaN,NaN,NaN,...,f,f,1.0,0.0,1.0,0.0,NaN,37351.0,0.882391,272
49487,49487,11694,38059680.0,https://www.airbnb.com/rooms/38059680,2.020021e+13,2020-02-13,Cute two bedroom beach suite in Rockaway Beach,This is a cute little guest suite just three h...,Our cute little space is just steps from the b...,This is a cute little guest suite just three h...,...,f,f,1.0,1.0,0.0,0.0,1.90,37351.0,0.882391,272
49488,49488,11697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39540.0,0.955157,4
50262,50262,10174,20933849.0,https://www.airbnb.com/rooms/20933849,2.020021e+13,2020-02-12,the best you can find,NaN,NaN,NaN,...,f,f,1.0,1.0,0.0,0.0,NaN,NaN,NaN,9


In [266]:
listings_with_IncEduCri_NYC.to_csv('listings_with_IncEduCri_NYC.csv')